In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tseries.offsets import DateOffset
from tqdm import tqdm
import statistics as stat
import math


# Data Reading

First, we have to create dataframes from the given datasets. We have to sort them, clean them, and combine the relevant data.

In [3]:
# Creating dataframe with monthly data for companies
monthly_dataset = pd.read_csv("NL_FR_BE_data_monthly.csv")
monthly_dataset['mdate'] = pd.to_datetime(monthly_dataset['mdate'], format="%Y%m")
monthly_dataset["ISIN"] = monthly_dataset["ISIN"].astype(str) 
monthly_dataset = monthly_dataset.sort_values(["ISIN", "mdate"], ascending=[True, True])       # Sort data by security and date, for easier access
monthly_dataset = monthly_dataset.reset_index()
monthly_dataset = monthly_dataset.drop(["index"], axis = 1)   # Delete duplicate column
monthly_dataset = monthly_dataset.dropna()                            # Just to be sure
print(monthly_dataset.head(3).to_markdown(), "\n")

# Creating annual dataframe for companies
# First we read the given annual data
annual_dataset = pd.read_csv("NL_FR_BE_data_annual.csv")
annual_dataset['fyear'] = pd.to_datetime(annual_dataset['fyear'], format="%Y") + DateOffset(months=5)       # FF data is starting from June
annual_dataset["ISIN"] = annual_dataset["ISIN"].astype(str) 
annual_dataset = annual_dataset.sort_values(["ISIN", "fyear"], ascending=[True, True])       # Sort data by security and date, for easier access
annual_dataset = annual_dataset.reset_index()
annual_dataset = annual_dataset.drop(["index"], axis = 1)                                   # Delete duplicate column
# Next we get relevant data for annual dataframe from the monthly one
monthly_dataset_annual = monthly_dataset.copy()
mask = monthly_dataset_annual['mdate'].dt.month == 6
monthly_dataset_annual = monthly_dataset_annual.loc[mask, :]
monthly_dataset_annual["RET10"] =  monthly_dataset_annual["RET11"] - monthly_dataset_annual["RET"]      # Deleting last month's returns to avoid short term mean-reversion
monthly_dataset_annual["fyear"] = monthly_dataset_annual["mdate"]
monthly_dataset_annual = monthly_dataset_annual.drop(["RET11", "RET", "mdate"], axis = 1)   # Delete duplicate
annual_dataset = annual_dataset.merge(monthly_dataset_annual)       # Final dataset on which we can trade
annual_dataset = annual_dataset.dropna()                            # Just to be sure
print(annual_dataset.head(3).to_markdown(), "\n")

# Finally creating monthly dataframe of the market
monthly_fixed_params = pd.read_csv("Europe_FF_Factors.csv")
monthly_fixed_params['mdate'] = pd.to_datetime(monthly_fixed_params['mdate'], format="%Y%m")
monthly_fixed_params = monthly_fixed_params.sort_values(["mdate"], ascending=[True])       # Sort data by security and date, for easier access
monthly_fixed_params = monthly_fixed_params.reset_index()
monthly_fixed_params = monthly_fixed_params.drop(["index"], axis = 1)   # Delete duplicate column
# Parameters that will be later used to trade (Filters for Signals based on FF)
monthly_fixed_params["b_border"] = np.zeros(len(monthly_fixed_params))
monthly_fixed_params["h_border"] = np.zeros(len(monthly_fixed_params))
monthly_fixed_params["s_border"] = np.zeros(len(monthly_fixed_params))
monthly_fixed_params["ivol_border"] = np.zeros(len(monthly_fixed_params))
print(monthly_fixed_params.head(3).to_markdown(), "\n")

|    | ISIN         | mdate               |     RET |   RET11 |      ME |      b |       h |       s |   ivol |
|---:|:-------------|:--------------------|--------:|--------:|--------:|-------:|--------:|--------:|-------:|
|  0 | ANN4327C1220 | 1991-06-01 00:00:00 | -0.0256 | -0.2089 | 712.452 | 1.2541 | -0.9141 | -0.1884 | 0.0074 |
|  1 | ANN4327C1220 | 1991-07-01 00:00:00 |  0.024  |  0.1856 | 729.56  | 1.1767 | -1.2208 | -0.2632 | 0.0075 |
|  2 | ANN4327C1220 | 1991-08-01 00:00:00 | -0.0799 |  0.2478 | 671.284 | 0.9473 | -1.7583 | -0.5612 | 0.0189 | 

|    | ISIN         | fyear               |   BEME |     OP |     INV |      ME |      b |       h |       s |   ivol |   RET10 |
|---:|:-------------|:--------------------|-------:|-------:|--------:|--------:|-------:|--------:|--------:|-------:|--------:|
|  0 | ANN4327C1220 | 1991-06-01 00:00:00 | 0.589  | 0.5048 |  0.115  | 712.452 | 1.2541 | -0.9141 | -0.1884 | 0.0074 | -0.1833 |
|  1 | ANN4327C1220 | 1992-06-01 00:00:00 | 0.72

Possibilities for Trades:
1) We are scored based on the FF-3 outperformance. To us is given FF-3, with WML (more on it here https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_developed_mom.html).
2) With given data we can find FF-5, because Investment(INV) and Operating Profitability (OP) are given (However, only annually)
3) Also, with given data we can easily find momentum, because Trailing Returns are given (Ret11, Ret10)
4) Moreover, implied volatility of returns is given, which means we can create volatility filter too. Combine with some Risk Free rate etc.
5) Currently we have implemented a simple strategyh which trades on the given FF-3 factors. We can improve a lot here. It is also running for around 10min overall

# Strategy
We are trading based on the given FF-3 factors and implied volatility. If company is high on all 3 residuals and has high volatility of returns, we buy it

In [4]:
# Generating Signals

start_date = monthly_dataset["mdate"].min()
end_date = monthly_dataset["mdate"].max()
datelist = pd.date_range(start_date, end_date, freq ='MS').tolist()
i=-1

# First we have to calculate the border of FF-3 parameters to trade on them. It is just a higher half in our case.
for date in datelist:
    i+=1    # Counter to access correct point in dataframe
    monthly_param_dataset = monthly_dataset[monthly_dataset["mdate"] == date]   # Get all the stocks for the given date
    # Finding borders for residuals, based on all the company residuals of that month
    monthly_fixed_params.iloc[i, monthly_fixed_params.columns.get_loc("b_border")] = monthly_param_dataset["b"].quantile(q=0.5)
    monthly_fixed_params.iloc[i, monthly_fixed_params.columns.get_loc("h_border")] = monthly_param_dataset["h"].quantile(q=0.5)
    monthly_fixed_params.iloc[i, monthly_fixed_params.columns.get_loc("s_border")] = monthly_param_dataset["s"].quantile(q=0.5)
    monthly_fixed_params.iloc[i, monthly_fixed_params.columns.get_loc("ivol_border")] = monthly_param_dataset["ivol"].quantile(q=0.5)

print(monthly_fixed_params.head(3).to_markdown(), "\n")

|    | mdate               |   MktRF |     SMB |     HML |     RF |    WML |   b_border |   h_border |   s_border |   ivol_border |
|---:|:--------------------|--------:|--------:|--------:|-------:|-------:|-----------:|-----------:|-----------:|--------------:|
|  0 | 1991-06-01 00:00:00 | -0.0741 |  0.0061 | -0.0069 | 0.0042 | 0.0054 |    1.07925 |    -0.083  |     0.3317 |       0.0132  |
|  1 | 1991-07-01 00:00:00 |  0.053  | -0.0323 | -0.0035 | 0.0049 | 0.0506 |    1.08765 |    -0.1328 |     0.3626 |       0.01275 |
|  2 | 1991-08-01 00:00:00 |  0.0142 | -0.0093 | -0.0008 | 0.0046 | 0.0377 |    1.0084  |    -0.0946 |     0.3488 |       0.0134  | 



Now we can setup trades. We parse all companies and check for each month if their residuals are higher than border

In [5]:
# Marking Trade
monthly_dataset["Trade_Status"] = np.zeros(len(monthly_dataset))    # Creating empty column to hold trade status (1: long, -1: short, 0: None)
i=-1

# Getting long trades
for date in tqdm(datelist):                             # Combining entries by month to increase the speed (tqdm is a fancy progress bar)
    i+=1                                                # Counter to access correct fixed month values
    month_profile = monthly_fixed_params.iloc[i]        # Getting months results from month_fixed_data
    date_mask = monthly_dataset["mdate"] == date        # Masking only the relevant months
    complete_mask = np.zeros(len(monthly_dataset))      # Complete final mask which uses all the signals created earlier
    monthly_dataset["flag1"] = monthly_dataset.loc[date_mask]["b"] > month_profile["b_border"]          # Checking each signal one by one
    monthly_dataset["flag2"] = monthly_dataset.loc[date_mask]["h"] > month_profile["h_border"]
    monthly_dataset["flag3"] = monthly_dataset.loc[date_mask]["s"] > month_profile["s_border"]
    monthly_dataset["flag4"] = monthly_dataset.loc[date_mask]["ivol"] > month_profile["ivol_border"]
    complete_mask = monthly_dataset["flag1"] * monthly_dataset["flag2"] * monthly_dataset["flag3"] * monthly_dataset["flag4"]       # Combining signals
    complete_mask = complete_mask.shift(1)              # We have to shift result, so that we are not trading on past data
    complete_mask = complete_mask.fillna(0)             # Filling empty spaces with 0
    monthly_dataset["Trade_Status"] += complete_mask     # Setting trade status

# # Getting short trades (Another signals and inverted mask, because shorts have a value of -1 in dataframe)
# for date in tqdm(datelist):                             # Combining entries by month to increase the speed (tqdm is a fancy progress bar)
#     i+=1                                                # Counter to access correct fixed month values
#     month_profile = monthly_fixed_params.iloc[i]        # Getting months results from month_fixed_data
#     date_mask = monthly_dataset["mdate"] == date        # Masking only the relevant months
#     complete_mask = np.zeros(len(monthly_dataset))      # Complete final mask which uses all the signals created earlier
#     monthly_dataset["flag1"] = monthly_dataset.loc[date_mask]["b"] < month_profile["b_border"]          # Checking each signal one by one
#     monthly_dataset["flag2"] = monthly_dataset.loc[date_mask]["h"] < month_profile["h_border"]
#     monthly_dataset["flag3"] = monthly_dataset.loc[date_mask]["s"] < month_profile["s_border"]
#     monthly_dataset["flag4"] = monthly_dataset.loc[date_mask]["ivol"] < month_profile["ivol_border"]
#     complete_mask = monthly_dataset["flag1"] * monthly_dataset["flag2"] * monthly_dataset["flag3"] * monthly_dataset["flag4"]       # Combining signals
#     complete_mask = complete_mask.shift(1)              # We have to shift result, so that we are not trading on past data
#     complete_mask = complete_mask.fillna(0)             # Filling empty spaces with 0
#     monthly_dataset["Trade_Status"] = -complete_mask     # Setting trade status

100%|██████████| 349/349 [00:48<00:00,  7.21it/s]


In [6]:
# Calculating Returns

# Creating dataframe with final returns
monthly_returns = pd.DataFrame()
monthly_returns["mdate"] = monthly_fixed_params["mdate"].dt.date
monthly_returns["long_returns"] = np.zeros(len(monthly_returns))
monthly_returns["long_positions"] = np.zeros(len(monthly_returns))
monthly_returns["short_returns"] = np.zeros(len(monthly_returns))
monthly_returns["short_positions"] = np.zeros(len(monthly_returns))

i=-1
for date in tqdm(datelist):                             # Combining entries by month to increase the speed (tqdm is a fancy progress bar)
    i+=1                                                # Counter to access correct fixed month values
    date_mask = monthly_dataset["mdate"] == date        # Masking only the relevant month
    long_trade_mask = monthly_dataset["Trade_Status"] > 0
    short_trade_mask = monthly_dataset["Trade_Status"] < 0
    if len(monthly_dataset[date_mask & long_trade_mask]) > 0:
        monthly_returns["long_returns"].iloc[i] = monthly_dataset[date_mask & long_trade_mask]["RET"].sum()
        monthly_returns["long_positions"].iloc[i] = len(monthly_dataset[date_mask & long_trade_mask]["RET"])
    if len(monthly_dataset[date_mask & short_trade_mask]) > 0:
        monthly_returns["short_returns"].iloc[i] = monthly_dataset[date_mask & short_trade_mask]["RET"].sum()
        monthly_returns["short_positions"].iloc[i] = len(monthly_dataset[date_mask & short_trade_mask]["RET"])

  0%|          | 0/349 [00:00<?, ?it/s]/tmp/ipykernel_8863/2531029596.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_returns["long_returns"].iloc[i] = monthly_dataset[date_mask & long_trade_mask]["RET"].sum()
/tmp/ipykernel_8863/2531029596.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_returns["long_positions"].iloc[i] = len(monthly_dataset[date_mask & long_trade_mask]["RET"])
100%|██████████| 349/349 [00:00<00:00, 466.84it/s]


In [7]:
print(monthly_returns.to_markdown())
print("\nFinal Results:\n")
# ONLY LONG FOR NOW
print("Strategy Return (Only): ", (monthly_returns["long_returns"]/monthly_returns["long_positions"]).sum())      # Equal weights are assumed
print("Strategy STD (Only): ", (monthly_returns["long_returns"]/monthly_returns["long_positions"]).std())      # Equal weights are assumed
print("Market Return: ", monthly_fixed_params["MktRF"].sum())
print("Risk Free Return: ", monthly_fixed_params["RF"].sum())

|     | mdate      |   long_returns |   long_positions |   short_returns |   short_positions |
|----:|:-----------|---------------:|-----------------:|----------------:|------------------:|
|   0 | 1991-06-01 |        -1.2688 |               32 |               0 |                 0 |
|   1 | 1991-07-01 |         0.143  |               33 |               0 |                 0 |
|   2 | 1991-08-01 |         0.6178 |               29 |               0 |                 0 |
|   3 | 1991-09-01 |         0.6388 |               28 |               0 |                 0 |
|   4 | 1991-10-01 |         0.1283 |               26 |               0 |                 0 |
|   5 | 1991-11-01 |        -1.4079 |               35 |               0 |                 0 |
|   6 | 1991-12-01 |         1.4461 |               30 |               0 |                 0 |
|   7 | 1992-01-01 |         0.5508 |               36 |               0 |                 0 |
|   8 | 1992-02-01 |        -0.0379 |             

In [8]:
# Sharpe ratio
exc_ret = monthly_returns["long_returns"].sum() - (monthly_fixed_params["RF"]).sum()
exc_ret_ann = exc_ret/len(monthly_returns["long_returns"]) * np.sqrt(12)
#exc_ret_ann = (250 / len(monthly_returns["long_returns"])) * exc_ret
std = np.std(monthly_returns["long_returns"] - monthly_fixed_params["RF"])
#std_ann = math.sqrt(250 / len(monthly_returns["long_returns"])) * std

SR_ann = exc_ret_ann / std

print(f"Annual SR: {SR_ann}")

# Info ratio
exc_ret_b = monthly_returns["long_returns"].sum() - (monthly_fixed_params["MktRF"]).sum()
exc_ret_b_ann = (250 / len(monthly_returns["long_returns"])) * exc_ret_b
std_b = np.std(monthly_returns["long_returns"] - monthly_fixed_params["MktRF"])
std_b_ann = math.sqrt(250 / len(monthly_returns["long_returns"])) * std_b

IR_ann = exc_ret_b_ann / std_b_ann

print(f"Annual IR: {IR_ann}")

Annual SR: 0.5493274599303344
Annual IR: 47.03593841046036
